In [17]:
!pip install torchaudio==0.13.0 torchvision==0.14.0 torch==1.13.0


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [18]:
!pip install openmim==0.3.2
!mim install mmcv-full==1.7.0
!pip install -e .


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.0/index.html

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Obtaining file:///workspace
ERROR: file:///workspace does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
!pip install -r requirements.txt 

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:
!apt install libgl1-mesa-glx --yes

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


In [22]:
!pip install mmdet==2.27.0


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [24]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from mmdet.models import build_detector
from mmdet.apis import init_detector, inference_detector
from mmcv import Config

In [25]:
model_path = "best_bbox_h_mAP_epoch_33.pth" 
config_path = "deformable_detr_config.py"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = init_detector(config_path, model_path, device=device)

cuda
load checkpoint from local path: best_bbox_h_mAP_epoch_33.pth


In [26]:
frame = cv2.imread("20241203_170425.jpg")  # Replace with an actual frame
results = inference_detector(model, frame)
print(f"Inference results: {results}")


Inference results: [array([[2.0831074e+03, 9.6526465e+02, 2.8128628e+03, 2.5641741e+03,
        4.9368867e-01],
       [3.3845750e+03, 1.0645931e+03, 3.9273647e+03, 1.6189938e+03,
        1.2651945e-02],
       [3.3891370e+03, 1.0350955e+03, 3.9873254e+03, 2.4602876e+03,
        1.1400951e-02],
       [3.3897900e+03, 1.0425740e+03, 3.9155366e+03, 1.5803932e+03,
        8.9927604e-03],
       [3.4028230e+03, 1.0022028e+03, 3.9817976e+03, 2.2375610e+03,
        7.8201238e-03],
       [3.3930881e+03, 1.1216971e+03, 3.9896345e+03, 2.5940471e+03,
        7.7070375e-03],
       [3.3110276e+03, 1.0868293e+03, 3.8218044e+03, 2.4393025e+03,
        6.3802232e-03],
       [2.1897258e+03, 9.8227582e+02, 2.7687839e+03, 2.0344824e+03,
        5.0302655e-03],
       [3.4283547e+03, 9.9680493e+02, 3.8611328e+03, 1.6395022e+03,
        4.6840161e-03],
       [3.4293948e+02, 1.9060269e+02, 1.2777229e+03, 1.5074236e+03,
        4.2287335e-03],
       [3.3448044e+03, 4.9189526e+02, 3.7638420e+03, 1.66992

In [49]:
import cv2
cap = cv2.VideoCapture("output.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

video = []

while True:    
    rt, frame = cap.read()
    if not rt:
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (640, 480))

    video.append(frame_resized)

#print(video)

30.0


In [50]:
from time import sleep
from tqdm import tqdm
import cv2

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
saved_video = cv2.VideoWriter("saved_video.mp4", fourcc, fps, (640, 480))

for frame_resized in tqdm(video):
    results = inference_detector(model, frame_resized)

    #print(f"Results length: {len(results)}")
    for idx, result in enumerate(results):
        #print(f"Processing result {idx+1}/{len(results)}")
        for bbox in result:
            #print("Bounding box:", bbox)
            x_min, y_min, x_max, y_max, score = bbox
            if score > 0.95:  # Confidence threshold
                label = "Gun Detected"
                #print(label)
                cv2.rectangle(frame_resized, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                #cvzone.cornerRect(frame, [int(x1), int(y1), int(x2-x1), int(y2-y1)], l=9, rt=3)
                cv2.putText(frame_resized, label, (int(x_min), int(y_min)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    
    frame_resized = frame_resized[:, :, ::-1]
    saved_video.write(frame_resized)

saved_video.release()
cv2.destroyAllWindows()

print("done")

100%|██████████| 2606/2606 [02:10<00:00, 20.04it/s]

done
